In [1]:
import pandas as pd
import numpy as np

In [2]:
og_df = pd.read_csv('../data/retail_data.csv')

og_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### Minimum restrictions

In [3]:
# ignoring entries with quantity or unitprice less than 0 (returns)
df_minimums = og_df[(og_df['Quantity'] > 0) & (og_df['UnitPrice'] > 0)].copy()

print('Minimun Quantity in dataframe: ', df_minimums.Quantity.min())
print('Minimun UnitPrice in dataframe: ', df_minimums.UnitPrice.min())

Minimun Quantity in dataframe:  1
Minimun UnitPrice in dataframe:  0.001


In [4]:
df_minimums.to_csv('../data/retail_0minimums.csv')

### Totals

In [5]:
# grouping CustomerID, Country, InvoiceDate
# summing UnitPrice to get TotalPrice, summing quantity
# ignoring entries with quantity or unitprice less than 0 (returns)
df_totals = df_minimums
df_totals['TotalPrice'] = df_minimums['Quantity'] * df_minimums['UnitPrice']
df_totals = df_totals.groupby(['CustomerID', 'InvoiceDate', 'Country']).agg({'Quantity': 'sum', 'StockCode': 'nunique', 'TotalPrice': 'sum'}).reset_index()
df_totals.rename(columns={'Quantity': 'TotalQuantity', 'StockCode': 'UniqueStockCodes'}, inplace=True)

df_totals.head()

,CustomerID,InvoiceDate,Country,TotalQuantity,UniqueStockCodes,TotalPrice
0,12346.0,1/18/2011 10:01,United Kingdom,74215,1,77183.60
1,12347.0,1/26/2011 14:30,Iceland,315,29,475.39
2,12347.0,10/31/2011 12:25,Iceland,676,47,1294.32
3,12347.0,12/7/2010 14:57,Iceland,319,31,711.79
4,12347.0,12/7/2011 15:52,Iceland,192,11,224.82


In [6]:
df_totals.to_csv('../data/retail_totals.csv')

### Removing Outliers

#### Totals

In [16]:
# IQR method
def identify_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (series < lower_bound) | (series > upper_bound)

# excluding outliers based on TotalPrice
outliers_mask = identify_outliers(df_totals['TotalPrice'])
df_iqr_totals = df_totals[~outliers_mask]


In [17]:
print('min total price: ', df_iqr_totals.TotalPrice.min())
print('max total price: ', df_iqr_totals.TotalPrice.max())
print()
print('Column and Rows before outlier removale: ', df_totals.shape)
print('Column and Rows after outlier removale: ', df_iqr_totals.shape)
     
df_iqr_totals.head()

min total price:  0.38
max total price:  946.41

Column and Rows before outlier removale:  (18440, 6)
Column and Rows after outlier removale:  (16978, 6)


,CustomerID,InvoiceDate,Country,TotalQuantity,UniqueStockCodes,TotalPrice
1,12347.0,1/26/2011 14:30,Iceland,315,29,475.39
3,12347.0,12/7/2010 14:57,Iceland,319,31,711.79
4,12347.0,12/7/2011 15:52,Iceland,192,11,224.82
5,12347.0,4/7/2011 10:43,Iceland,483,24,636.25
6,12347.0,6/9/2011 13:01,Iceland,196,18,382.52


In [18]:
df_iqr_totals.to_csv('../data/iqr_totals.csv')

#### Minimum Restriction

In [14]:
# removing outliers based on UnitPrice
# could also try basing outliers on quantity
outliers_mask = identify_outliers(df_minimums['UnitPrice'])
df_iqr = df_minimums[~outliers_mask]

print('min total price: ', df_iqr.UnitPrice.min())
print('max total price: ', df_iqr.UnitPrice.max())
print()
print('Column and Rows before outlier removale: ', df_minimums.shape)
print('Column and Rows after outlier removale: ', df_iqr.shape)
     
df_iqr.head()

min total price:  0.001
max total price:  8.33

Column and Rows before outlier removale:  (530104, 9)
Column and Rows after outlier removale:  (492105, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [19]:
df_iqr.to_csv('../data/iqr.csv')